# A-01-01B: Understanding Datasets - SMUD Billing Data

## 1. Import Libraries required

### 1.1 Basic Libraries

In [1]:
import os
import sys
import importlib
import pathlib as pl
import pandas as pd
import dask.dataframe as dd
import multiprocessing

### 1.2 Additional Libraries used only in this script

In [2]:
import re
import numpy as np

### 1.3 Load User-Written Libraries

In [3]:
# # 1. To generate objects that will be used to load user-written libraries
# ## Note: The numbers used below depend on the directory in which this script is
cwd = pl.Path.cwd()
path_project = cwd.parents[3]
path_hd = cwd.parents[2]
sys.path.append(str(path_hd))
os.chdir(path_project)

In [4]:
# # 2. To load user-written libraries
HD = importlib.import_module('H-Energy-Demand-Analysis')
FNC = importlib.import_module('F-Energy-Demand-Analysis_Common-Functions')
DD = importlib.import_module('D-Energy-Demand-Analysis_Data-Dictionary')

## 2. Set Path(s) and Parameter(s)

### 2.1 Set Path(s)

In [5]:
# # 1. Path(s) from which data file(s) will be loaded
FILE_TO_LOAD = 'SMUD_Billing-Data.parquet'
PATH_TO_LOAD = os.path.join(HD.PATH_DATA_INTERMEDIATE_SMUD_BILLING, FILE_TO_LOAD)

# # 2. Path(s) at which data file(s) will be saved
# (NOT Applicable)

### 2.2 Set Parameter(s)

In [6]:
# # 0. Basic Parameters
# # 0.1. Script Number
SCRIPT_NO = 'B-01-01B'
# # 0.2. Parameter for Dask DF
N_PARTITIONS = multiprocessing.cpu_count()

## 3. Details of Work

### 3.1. Load Dataset(s) required

In [7]:
smud_billing = dd.read_parquet(PATH_TO_LOAD)

### 3.2. Get Basic Information about the Dataset

#### 3.2.1. Check Data Types

In [8]:
FNC.ddf_info(smud_billing)

<class 'dask.dataframe.core.DataFrame'>
Columns: 18 entries, id_account to id_bu_part
dtypes: datetime64[ns](2), object(1), bool(1), float64(6), int32(7), int64(1)None


id_account                     int32
id_premise                     int32
is_pv                           bool
period_from           datetime64[ns]
period_to             datetime64[ns]
period_len                     int64
rate_code                     object
charge_fixed                 float64
charge_variable              float64
charge_total                 float64
kwh_total                      int32
kwh_t1                         int32
charge_variable_t1           float64
kwh_t2                         int32
charge_variable_t2           float64
kwh_t3                         int32
charge_variable_t3           float64
id_bu_part                     int32
dtype: object




#### 3.2.2. Find Primary Keys of the Dataset

In [9]:
# # 1) Check whether there are duplicated observations or not
test = smud_billing.shape[0] == smud_billing.drop_duplicates().shape[0]
test.compute()

True

The value of `True` means there are no duplicated observations.

In [10]:
# # 2) Find Primary Keys
list_cols_by \
    = ['id_account', 'id_premise', 'period_from', 'period_to', 'rate_code']
test = smud_billing.shape[0] \
        == smud_billing[list_cols_by].drop_duplicates().shape[0]
test.compute()

True

The values of `True` implies that the five data fields consist of primary keys of the DF.

### 3.3. Understand about the Dataset

#### 3.3.1. With respect to `period_from`, `period_to`, and `period_len`

In [11]:
# # 1) `period_from` <= `period_to`?
test = smud_billing[smud_billing['period_len'] < 0].shape[0] == 0
test.compute()

True

The value of `True` demonstrates that `period_to` is greater than or equal to `period_from`.

In [12]:
# # 2) Distributions of `period_from`, `period_to`, and `period_len`?
smud_billing['period_from'].dt.year.value_counts(sort= False).apply('{0:,.0f}'.format, meta= ('period_from')).compute()

1999            1
2000            2
2001           11
2002           11
2003          194
2004      536,087
2005    6,067,969
2006    6,159,068
2007    6,200,358
2008    6,272,764
2009    6,283,470
2010    6,320,658
2011    6,334,873
2012    6,380,214
2013    5,942,169
dtype: object

In [13]:
smud_billing['period_to'].dt.year.value_counts(sort= False).apply('{0:,.0f}'.format, meta= ('period_to')).compute()

2004       43,577
2005    6,057,754
2006    6,152,890
2007    6,213,179
2008    6,276,466
2009    6,274,765
2010    6,302,647
2011    6,331,801
2012    6,393,777
2013    6,450,989
2014            4
dtype: object

In [14]:
smud_billing['period_len'].describe(percentiles= list(np.linspace(0, 1, 10, endpoint= False))).apply('{0:,.1f}'.format, meta= ('period_len')).compute()

count    56,497,849.0
mean             29.0
std               6.2
min               0.0
0%                0.0
10%              28.0
20%              28.0
30%              28.0
40%              29.0
50%              29.0
60%              30.0
70%              30.0
80%              31.0
90%              32.0
max           3,269.0
dtype: object

The three tables above show:
 + Although there are observations before 2004 for `period_from`, there are no observations before 2004 for `period_to`.
 + There are observations having significantly long billing period.
 + There are observations with `period_len == 0`.

In [15]:
# # 3) Overlapped periods?
# ## Make temporary data fields
smud_billing['tmp_id_account'] = smud_billing['id_account'].shift()
smud_billing['tmp_id_premise'] = smud_billing['id_premise'].shift()
smud_billing['tmp_period_to'] = smud_billing['period_to'].shift()


In [16]:
# ## (Continued ...)
smud_billing.query('period_from <= tmp_period_to & id_account == tmp_id_account & id_premise == tmp_id_premise').shape[0].compute()

123

There are overlapped billing periods for customers having the same `id_account` and `id_premise`.

#### 3.3.2. With respect to `charge_*`s and `kwh_*`s

In [17]:
# # 1) "Tier 3" exist?
smud_billing.loc[smud_billing['charge_variable_t3'] > 0, 'period_from'].dt.year.value_counts(sort= False).apply('{0:,.0f}'.format, meta= ('period_from')).compute()

2003            1
2004      184,468
2005    1,500,921
2006    1,657,693
2007    1,539,329
2008    1,589,161
2009    1,050,425
dtype: object

In [18]:
# ## (Continued ...)
smud_billing.loc[smud_billing['charge_variable_t3'] > 0, 'period_to'].dt.year.value_counts(sort= False).apply('{0:,.0f}'.format, meta= ('period_to')).compute()

2004        7,852
2005    1,515,365
2006    1,630,017
2007    1,545,346
2008    1,593,637
2009    1,229,778
2010            3
dtype: object

In [19]:
# # 2) Summation Checks
select = smud_billing['charge_total'] == smud_billing['charge_fixed'] + smud_billing['charge_variable']
N_hold = smud_billing[select].shape[0]
N = smud_billing.shape[0]
share_hold = N_hold / N * 100
print('{0:,.2f}'.format(share_hold.compute()) + '%')

1.65%


In [20]:
# ## (Continued ...)
select = smud_billing['charge_variable'] == smud_billing['charge_variable_t1'] + smud_billing['charge_variable_t2'] + smud_billing['charge_variable_t3']
N_hold = smud_billing[select].shape[0]
N = smud_billing.shape[0]
share_hold = N_hold / N * 100
print('{0:,.2f}'.format(share_hold.compute()) + '%')

87.10%


In [21]:
# ## (Continued ...)
select = smud_billing['kwh_total'] == smud_billing['kwh_t1'] + smud_billing['kwh_t2'] + smud_billing['kwh_t3']
N_hold = smud_billing[select].shape[0]
N = smud_billing.shape[0]
share_hold = N_hold / N * 100
print('{0:,.2f}'.format(share_hold.compute()) + '%')

99.96%


The results shows several interesting points:
+ The relationship `charge_total == charge_fixed + charge_variable` does NOT hold, except small number of observations.
+ There are observation not to satisfy the relationship `charge_variable == charge_variable_t1 + charge_variable_t2 + charge_variable_t3`.
+ The relationship `kwh_total == kwh_t1 + kwh_t2 + kwh_t3` generally holds.

More study on how the amount of total charge is determined is necessary.

#### 3.3.3. With respect to `id_bu_part`

In [22]:
list_cols_by = ['id_account', 'id_premise', 'rate_code']
test = smud_billing[list_cols_by].drop_duplicates().shape[0] == smud_billing[list_cols_by + ['id_bu_part']].drop_duplicates().shape[0]
test.compute()

True

The value of `True` implies that `id_bu_part` is unique given `id_account`, `id_premise`, and `rate_code`.